In [1]:
import os
os.environ['NOTEBOOK_MODE'] = '1'
import sys
import torch as ch
import numpy as np
import seaborn as sns
from PIL import Image
from scipy import stats
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
from robustness import model_utils, datasets
from robustness.tools.vis_tools import show_image_row, show_image_column
from robustness.tools.constants import CLASS_DICT
from pathlib import Path
import json
sys.path.append('./cycle_gan')
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html, util
import argparse
%matplotlib inline

# Constants
CONFIG = 'configs/places_room.json'
DATA = 'PlacesRoom' 
BATCH_SIZE = 4
NUM_WORKERS = 8
NOISE_SCALE = 20

DATA_SHAPE = 224 # Image size (fixed for dataset)
REPRESENTATION_SIZE = 512 # Size of representation vector (fixed for model)

# Load dataset
file = open(Path('robustness').joinpath(CONFIG))
cfg = json.load(file)
file.close()

dataset_function = getattr(datasets, DATA)
dataset = dataset_function(cfg['data'])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS, 
                                      batch_size=BATCH_SIZE // 2, 
                                      data_aug=False)
data_iterator = enumerate(test_loader)

# Load Model
model_kwargs = {
    'arch': 'resnet18',
    'dataset': dataset,
    'resume_path': '/data/theory/robustopt/aajay/models/Places_room_adv.pt',
    'old_format': False,
}
model_kwargs['state_dict_path'] = 'model'
model, _ = model_utils.make_and_restore_model(**model_kwargs)
model.eval()

# Custom loss to maximize a given component of the representation vector
def feature_vis_loss(model, inp, targ):
    _, rep = model(inp, 
                   with_latent=True, 
                   fake_relu=True) #To propagate gradients through the "zero-region" of a ReLU activation
    loss = rep[:, targ]
    return loss, None

# PGD parameters
kwargs = {
    'criterion': ch.nn.CrossEntropyLoss(),
    'custom_loss': feature_vis_loss,
    'constraint':'2',
    'eps':70,
    'step_size': 1,
    'iterations': 200, 
    'use_best': False
}

Img_choice_to_path = {'study room':'./demo_inputs/study_room.jpg',
                      'living room':'./demo_inputs/living_room.jpg', 
                      'reading room':'./demo_inputs/reading_room.jpg', 
                      'hotel room':'./demo_inputs/hotel_room2.jpg'}

activation_choice_to_num = {'living room 1':146,
                            'living room 2':321,
                            'formal room':68,
                            'meeting room':296,
                            'conference room':338}

filter_type = {'sepia':'hotel_cyclegan', 'blue':'hotel2_cyclegan'}

==> Preparing dataset places_room..
=> loading checkpoint '/data/theory/robustopt/aajay/models/Places_room_adv.pt'
=> loaded checkpoint '/data/theory/robustopt/aajay/models/Places_room_adv.pt' (epoch 105)


In [2]:
# Function definitions

def modify_image(src_img, dst_type, model):
    im = ch.tensor(np.transpose(src_img / 255., (0, 3, 1, 2))).float()
    act = activation_choice_to_num[dst_type]
    _, xadv = model(im.clone(), act, make_adv=True, **kwargs)
    return np.transpose(xadv.detach().cpu().numpy(), axes=(0,2,3,1))

def add_filter_image(src_img, filter_color):
    model_name = filter_type[filter_color]
    opt = argparse.Namespace(aspect_ratio=1.0, 
                             batch_size=1, 
                             checkpoints_dir='./cycle_gan/checkpoints', 
                             crop_size=256, dataroot='./cycle_gan/datasets', 
                             dataset_mode='single', 
                             direction='AtoB', 
                             display_id=-1, 
                             display_winsize=256, 
                             epoch='latest', 
                             eval=False, 
                             gpu_ids=[0], 
                             init_gain=0.02, 
                             init_type='normal', 
                             input_nc=3, 
                             isTrain=False, 
                             load_iter=0, 
                             load_size=256, 
                             max_dataset_size=float("inf"), 
                             model='test', 
                             model_suffix='', 
                             n_layers_D=3, 
                             name=model_name, 
                             ndf=64, 
                             netD='basic', 
                             netG='resnet_9blocks', 
                             ngf=64, 
                             no_dropout=True, 
                             no_flip=True, 
                             norm='instance', 
                             ntest=float("inf"), 
                             num_test=50, 
                             num_threads=0, 
                             output_nc=3, 
                             phase='test', 
                             preprocess='resize_and_crop', 
                             results_dir='./cycle_gan/sample_results/', 
                             serial_batches=True, suffix='', 
                             verbose=False)
    filter_model = create_model(opt)
    filter_model.setup(opt)         
    filter_model.eval()
    inp_img = ch.tensor(np.transpose(src_img, axes=(0,3,1,2))).float()
    filter_model.set_input({'A':inp_img, 'A_paths':None})
    filter_model.test()
    visuals = filter_model.get_current_visuals()
    visuals = util.tensor2im(visuals['fake_B'])[None]
    return visuals

def plot_image(src_img, dst_type, filter_color):
    img = np.asarray(Image.open(Img_choice_to_path[src_img]).resize((224, 224)))[None,:]
    mod_img = modify_image(img, dst_type, model)
    filter_mod_img = add_filter_image(mod_img, filter_color)
    print('Original Image')
    plt.imshow(img[0])
    plt.show()
    print('Modified Image')
    plt.imshow(mod_img[0])
    plt.show()
    print('Modified Image+Filter')
    plt.imshow(filter_mod_img[0])
    plt.show()

In [ ]:
# SOURCE IMAGE CHOICES: study room, living room, reading room, hotel room
# DESTINATION TYPE CHOICES: living room 1, living room 2, formal room, meeting room, conference room

plot_image(src_img='reading room', dst_type='living room 2', filter_color='sepia')